In [99]:
import pandas as pd
import requests
import spotipy as sp
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [100]:
#Credentials 
CLIENT_ID ='YOUR CLIENT ID'
CLIENT_SECRET = 'YOUR SECRET KEY'

#Authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

#POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

#Convert response to JSON
auth_response_data = auth_response.json()

#Store access token
access_token = auth_response_data['access_token']

In [101]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [102]:
#Convert JSON streaming data into pandas dataframes
df_stream0 = pd.read_json('StreamingHistory0.json')
df_stream1 = pd.read_json('streamingHistory1.json')
df_stream2 = pd.read_json('streamingHistory2.json')

# Combine streaming data files
df_stream = pd.concat([df_stream0, df_stream1, df_stream2])

# create a 'UniqueID' for each song by combining the fields 'artistName' and 'trackName'
df_stream['UniqueID'] = df_stream['artistName'] + ":" + df_stream['trackName']
df_stream


,endTime,artistName,trackName,msPlayed,UniqueID
0,2020-12-03 16:18,Anderson .Paak,Winners Circle,4073,Anderson .Paak:Winners Circle
1,2020-12-04 15:13,KAMAUU,LăVĭNDŭR,238784,KAMAUU:LăVĭNDŭR
2,2020-12-04 15:17,Sweater Beats,Hey Ya,196147,Sweater Beats:Hey Ya
3,2020-12-04 15:19,Stromae,quand c'est ?,320,Stromae:quand c'est ?
4,2020-12-04 15:19,Ruth B.,Shadows,320,Ruth B.:Shadows
...,...,...,...,...,...
8007,2021-12-03 19:31,WizKid,Ginger (feat. Burna Boy),190,WizKid:Ginger (feat. Burna Boy)
8008,2021-12-03 19:34,Tems,Found (feat. Brent Faiyaz),209454,Tems:Found (feat. Brent Faiyaz)
8009,2021-12-03 19:37,WizKid,Ginger (feat. Burna Boy),196146,WizKid:Ginger (feat. Burna Boy)
8010,2021-12-03 19:42,WizKid,Essence (feat. Tems),248040,WizKid:Essence (feat. Tems)


In [103]:
#There was issues with reading the JSON file, so I created this line of code to help for the rest of the code.
with open('YourLibrary.json') as json_data:
    library_data = json.load(json_data)
library_data

{'tracks': [{'artist': 'Emmit Fenn',
   'album': 'Painting Greys',
   'track': 'Painting Greys',
   'uri': 'spotify:track:6kUohQ7py1XEcQAhUgf2Zz'},
  {'artist': 'Joeboy',
   'album': 'Baby',
   'track': 'Baby',
   'uri': 'spotify:track:02KGBPy9mVdvo4dPXxt3h5'},
  {'artist': 'Billie Eilish',
   'album': 'WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?',
   'track': "when the party's over",
   'uri': 'spotify:track:43zdsphuZLzwA9k4DJhU0I'},
  {'artist': 'Khalid',
   'album': 'American Teen',
   'track': 'Location',
   'uri': 'spotify:track:152lZdxL1OR0ZMW6KquMif'},
  {'artist': 'Frank Ocean',
   'album': 'Blonde',
   'track': 'Nights',
   'uri': 'spotify:track:7eqoqGkKwgOaWNNHx90uEZ'},
  {'artist': 'Disclosure',
   'album': 'Settle',
   'track': 'Latch',
   'uri': 'spotify:track:1DunhgeZSEgWiIYbHqXl0c'},
  {'artist': 'SBTRKT',
   'album': 'SBTRKT',
   'track': 'Hold On',
   'uri': 'spotify:track:3vlVbJmvSm3x5Hqmnzh8HI'},
  {'artist': 'Major Lazer',
   'album': 'Peace Is The Mission',
   'track'

In [115]:
#Streaming Library JSON to be converted into DataFrame Again with track uris (track links)
df_library = pd.DataFrame.from_dict(library_data, orient='index').T.reset_index()
df_library = pd.json_normalize(df_library['tracks'])
df_library.head()

,artist,album,track,uri
0,Emmit Fenn,Painting Greys,Painting Greys,spotify:track:6kUohQ7py1XEcQAhUgf2Zz
1,Joeboy,Baby,Baby,spotify:track:02KGBPy9mVdvo4dPXxt3h5
2,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",when the party's over,spotify:track:43zdsphuZLzwA9k4DJhU0I
3,Khalid,American Teen,Location,spotify:track:152lZdxL1OR0ZMW6KquMif
4,Frank Ocean,Blonde,Nights,spotify:track:7eqoqGkKwgOaWNNHx90uEZ


In [105]:
#Create a Unique ID to eventually connect the Data Library to the Pulled Data from the API
df_library['UniqueID'] = df_library['artist'] + ":" + df_library['track']
new = df_library["uri"].str.split(":", expand = True)
df_library['track_uri'] = new[2]
df_library.head()

,artist,album,track,uri,UniqueID,track_uri
0,Emmit Fenn,Painting Greys,Painting Greys,spotify:track:6kUohQ7py1XEcQAhUgf2Zz,Emmit Fenn:Painting Greys,6kUohQ7py1XEcQAhUgf2Zz
1,Joeboy,Baby,Baby,spotify:track:02KGBPy9mVdvo4dPXxt3h5,Joeboy:Baby,02KGBPy9mVdvo4dPXxt3h5
2,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",when the party's over,spotify:track:43zdsphuZLzwA9k4DJhU0I,Billie Eilish:when the party's over,43zdsphuZLzwA9k4DJhU0I
3,Khalid,American Teen,Location,spotify:track:152lZdxL1OR0ZMW6KquMif,Khalid:Location,152lZdxL1OR0ZMW6KquMif
4,Frank Ocean,Blonde,Nights,spotify:track:7eqoqGkKwgOaWNNHx90uEZ,Frank Ocean:Nights,7eqoqGkKwgOaWNNHx90uEZ


In [111]:
# Create a final copy of a dictionary, this allows to manipulate the data without editing the original data set. 
df_spotify_features = df_stream.copy()

# Do a left join with df_library using the UniqueID column and attach the album and track_uri
df_spotify_features = pd.merge(df_spotify_features, df_library[['album','UniqueID','uri','track_uri']],how='left',on=['UniqueID'])

# Remove songs not in the Library since those songs will not have track uris. 
df_spotify_features = df_spotify_features[df_spotify_features['uri'].notna()]

df_spotify_features.head()

,endTime,artistName,trackName,msPlayed,UniqueID,album,uri,track_uri
1,2020-12-04 15:13,KAMAUU,LăVĭNDŭR,238784,KAMAUU:LăVĭNDŭR,TheKAMAUU-CASSETTE: ŭRTH GōLD,spotify:track:5TfndFjdJY502LOhS4S9wX,5TfndFjdJY502LOhS4S9wX
4,2020-12-04 15:19,Ruth B.,Shadows,320,Ruth B.:Shadows,Dear White People Soundtrack Season 2 (A Netfl...,spotify:track:7AwXQNm39lNZV7MfEjK2LX,7AwXQNm39lNZV7MfEjK2LX
5,2020-12-04 15:19,Burna Boy,Anybody,24642,Burna Boy:Anybody,African Giant,spotify:track:44SSviC4R1TkAdsyptjDpE,44SSviC4R1TkAdsyptjDpE
6,2020-12-04 15:19,Jorja Smith,Be Honest (feat. Burna Boy),3583,Jorja Smith:Be Honest (feat. Burna Boy),Be Honest (feat. Burna Boy),spotify:track:5pAbCxt9e3f81lOmjIXwzd,5pAbCxt9e3f81lOmjIXwzd
8,2020-12-04 15:19,SPINALL,Nowo,25167,SPINALL:Nowo,Nowo,spotify:track:2twuWVm42q3E4okRqfv8U1,2twuWVm42q3E4okRqfv8U1


In [107]:
#Blank dictionary to store audio features from Spotify's API
feature_dict = {}

#Conevert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

#A for loop using the track URIs to pull audio features from the API
#Store it into a dictionary
for t_uri in track_uris:
    
    feature_dict[t_uri] = {'popularity': 0,
                           'danceability': 0,
                           'acousticness': 0,
                           'energy': 0,
                           'speechiness': 0,
                           'liveness': 0,
                           'loudness': 0,
                           'valence': 0,
                           'instrumentalness': 0,
                           'tempo': 0}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    feature_dict[t_uri]['popularity'] = r['popularity']
    
    #Audio features from Spotify's API
    s = requests.get(BASE_URL + 'audio-features/' + t_uri, headers=headers)
    s = s.json()
    feature_dict[t_uri]['acousticness'] = s['acousticness']
    feature_dict[t_uri]['danceability'] = s['danceability']
    feature_dict[t_uri]['energy'] = s['energy']
    feature_dict[t_uri]['speechiness'] = s['speechiness']
    feature_dict[t_uri]['liveness'] = s['liveness']
    feature_dict[t_uri]['loudness'] = s['loudness']
    feature_dict[t_uri]['valence'] = s['valence']
    feature_dict[t_uri]['instrumentalness'] = s['instrumentalness']
    feature_dict[t_uri]['tempo'] = s['tempo']

In [108]:
#Convert the new features dictionary into dataframe
df_features = pd.DataFrame.from_dict(feature_dict, orient='index')
df_features.insert(0, 'track_uri', df_features.index)
df_features.reset_index(inplace=True, drop=True)

df_features.head()

,track_uri,popularity,danceability,acousticness,energy,speechiness,liveness,loudness,valence,instrumentalness,tempo
0,6kUohQ7py1XEcQAhUgf2Zz,0,0.828,0.459,0.221,0.0838,0.0891,-14.332,0.200,0.033900,132.994
1,02KGBPy9mVdvo4dPXxt3h5,0,0.785,0.142,0.806,0.0558,0.1470,-5.340,0.539,0.000285,105.001
2,43zdsphuZLzwA9k4DJhU0I,85,0.367,0.978,0.111,0.0972,0.0897,-14.084,0.198,0.000040,82.642
3,152lZdxL1OR0ZMW6KquMif,81,0.736,0.330,0.449,0.4250,0.0898,-11.462,0.326,0.000162,80.126
4,7eqoqGkKwgOaWNNHx90uEZ,80,0.466,0.420,0.548,0.1180,0.1130,-9.362,0.423,0.000001,89.815


In [113]:
#Merge the audio features datafram with the spotify library data
spotify_my_data = pd.merge(df_features,df_spotify_features, on='track_uri')
spotify_my_data.head()

,track_uri,popularity,danceability,acousticness,energy,speechiness,liveness,loudness,valence,instrumentalness,tempo,endTime,artistName,trackName,msPlayed,UniqueID,album,uri
0,6kUohQ7py1XEcQAhUgf2Zz,0,0.828,0.459,0.221,0.0838,0.0891,-14.332,0.2,0.0339,132.994,2020-12-12 14:47,Emmit Fenn,Painting Greys,618,Emmit Fenn:Painting Greys,Painting Greys,spotify:track:6kUohQ7py1XEcQAhUgf2Zz
1,6kUohQ7py1XEcQAhUgf2Zz,0,0.828,0.459,0.221,0.0838,0.0891,-14.332,0.2,0.0339,132.994,2020-12-12 14:48,Emmit Fenn,Painting Greys,575,Emmit Fenn:Painting Greys,Painting Greys,spotify:track:6kUohQ7py1XEcQAhUgf2Zz
2,6kUohQ7py1XEcQAhUgf2Zz,0,0.828,0.459,0.221,0.0838,0.0891,-14.332,0.2,0.0339,132.994,2020-12-15 17:10,Emmit Fenn,Painting Greys,3349,Emmit Fenn:Painting Greys,Painting Greys,spotify:track:6kUohQ7py1XEcQAhUgf2Zz
3,6kUohQ7py1XEcQAhUgf2Zz,0,0.828,0.459,0.221,0.0838,0.0891,-14.332,0.2,0.0339,132.994,2020-12-20 15:32,Emmit Fenn,Painting Greys,3861,Emmit Fenn:Painting Greys,Painting Greys,spotify:track:6kUohQ7py1XEcQAhUgf2Zz
4,6kUohQ7py1XEcQAhUgf2Zz,0,0.828,0.459,0.221,0.0838,0.0891,-14.332,0.2,0.0339,132.994,2020-12-28 11:47,Emmit Fenn,Painting Greys,227368,Emmit Fenn:Painting Greys,Painting Greys,spotify:track:6kUohQ7py1XEcQAhUgf2Zz


In [110]:
#Save the final Spotify Data into a csv file
spotify_my_data.to_csv('MySpotifyData.csv')
print('done')

done
